## presequites

In [ ]:
# !pip install -U bitsandbytes flash_attn

In [ ]:
!pip install chromadb sentence_transformers 

In [ ]:
KOSMO_CHROMA_FOLDER = "PATH"

assert KOSMO_CHROMA_FOLDER != "PATH", "specify correct path"

chroma_client = chromadb.PersistentClient(path=KOSMO_CHROMA_FOLDER)

collection = chroma_client.get_or_create_collection(name="my_collection", 
                                                    metadata={"hnsw:space": "cosine"},
                                                    embedding_function=chromadb.utils.
                                                    embedding_functions.SentenceTransformerEmbeddingFunction(
                                                            model_name="sentence-transformers/distiluse-base-multilingual-cased-v2"
                                                        )
                                                   )

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM

tokenizer = AutoTokenizer.from_pretrained('NousResearch/Hermes-3-Llama-3.2-3B', trust_remote_code=True, device_map="auto")
model = LlamaForCausalLM.from_pretrained(
    "NousResearch/Hermes-3-Llama-3.2-3B",
    torch_dtype=torch.float16,
    device_map="auto"
)

In [ ]:
import chromadb
from itertools import count, islice

chroma_client = chromadb.PersistentClient(path="kosmo_chroma2")

# chroma_client.delete_collection(name="my_collection")

# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(name="my_collection", 
                                                    metadata={"hnsw:space": "cosine"},
                                                    embedding_function=chromadb.utils.
                                                    embedding_functions.SentenceTransformerEmbeddingFunction(
                                                            model_name="sentence-transformers/distiluse-base-multilingual-cased-v2"
                                                        )
                                                   )
# switch `add` to `upsert` to avoid adding the same documents every time

## working with model

In [ ]:
def get_augmentation_data(query: str, topk: int = 5, offset: int = 0) -> str:
    results = collection.query(
        query_texts=query, # Chroma will embed this for you
        n_results = topk + offset # how many results to return
    )

    return " ".join(results["documents"][0][offset:])

get_augmentation_data("Первый человек в космосе", 5, 4)

In [ ]:
def make_prompt(query: str) -> str:
    data = get_augmentation_data(query, 5, 4)
    sys_prompt = "<|im_start|>system \
    Ты разумный, развитый искусственный интеллект для ответа на любые вопросы пользователя касательно космонавтики. \
    Отвечай только на русском языке. Пользуйся информацией данной вместе с каждым вопросом<|im_end|>"

    return sys_prompt + "\n<|im_start|>user\n" + f"Используя информацию {data} ответь на вопрос {query}" + "<|im_end|>"


make_prompt("первый человек в космосе")

In [100]:
def talk_with_hermes(query: str) -> str:
    input_ids = tokenizer(make_prompt(query), return_tensors="pt").input_ids.to("cuda" if torch.cuda.is_available() else "cpu")
    generated_ids = model.generate(
        input_ids, max_new_tokens=500, 
        temperature=0.7, repetition_penalty=1.1, 
        do_sample=True, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id)
    response = tokenizer.decode(generated_ids[0][input_ids.shape[-1]:], skip_special_tokens=True, clean_up_tokenization_space=True)
    return response

query = "Кто конструктор ракеты Восток-1"

talk_with_hermes(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'\n<SOLUTION>Космическая программа "Восток" была разработана советским конструктором Сергеем Королевым. Он был и главным конструктором этого комплекса, который включал в себя ракету-носитель и первый модуль орбитальной станции «Салют-1». Ракета-восток была первой советской ракетой, способной вывести полезную нагрузку на низкую околоземную орбиту. Серия ракет Восток использовались для запуска космических кораблей «Восход», а также для доставки пилотируемых космических кораблей «Союз» к орбитальной станции.'

## db fillup

In [ ]:
!pip install pypdf

In [ ]:
import pypdf
import tqdm
import re

files = [
    '/kaggle/input/kosmospdf/Kosmonavtika_Kratkiy_spravochnik_Tom_1_Kosmonavtika_SSSR_i_Rossii.pdf',
    "/kaggle/input/kosmospdf/Kosmonavtika_Kratkiy_spravochnik_Tom_2_Kosmonavtika_SShA_2022.pdf",
    "/kaggle/input/kosmospdf/Kosmonavtika_Kratkiy_spravochnik_Tom_4_Mezhdunarodnye_proekty_2022.pdf",
    "/kaggle/input/kosmospdf/Kosmonavtika_Kratkiy_spravochnik_Tom_6_Kosmonavtika_v_tsifrakh_2022.pdf"
]

spaces = re.compile(" (\n| |\r|\r\n)+")
numbers = re.compile("[ ]?\d+.?")
dots = re.compile("\.{2,}")

idgen = count()

for file in tqdm.notebook.tqdm(files, position = 0):
    reader = pypdf.PdfReader(file)
    for page in tqdm.notebook.tqdm(reader.pages, position = 1):
        text = dots.sub("", numbers.sub(" ", spaces.sub(" ", page.extract_text().replace("\n", " ").strip())))
        if len(text) >= 60:
            collection.upsert(
                documents=text,
                ids=str(next(idgen))
            )
        